# Model Tuning
Grid Search Hyperparameter Tuning

In [1]:

# Reloading
%reload_ext autoreload
%autoreload 2

import pandas as pd
print('pandas version:{}'.format(pd.__version__))
import numpy as np
print('numpy version:{}'.format(np.__version__))
import math
from datetime import datetime
from dgpylib import dg_athena
from dgpylib import dg_s3
import os
import re
from matplotlib.ticker import MaxNLocator
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import sys
sys.path.insert(0, '../Modelling')
import helper_function2 as hf

import sys
sys.path.insert(0, '../EDA')
import data_integrity_fixer as dint
import basicstatsandplotter as bsp
import data_imputer as di
import data_integrity_fixer as dif
import parameters

##for modelling
import h2o
print('h2o version:{}'.format(h2o.__version__))
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators import H2OTargetEncoderEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.tree import H2OTree
from h2o.tree import H2ONode
from h2o.tree import H2OSplitNode
from h2o.tree import H2OLeafNode
from h2o.estimators.kmeans import H2OKMeansEstimator

#import xgboost as xgb
import sklearn
import category_encoders as ce
print('sklearn version:{}'.format(sklearn.__version__))
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score
from sklearn.preprocessing import PowerTransformer
import time
import parameters
from statsmodels.stats.outliers_influence import variance_inflation_factor

import math
from h2o.grid.grid_search import H2OGridSearch

%run 'functions.ipynb'

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100

pandas version:1.4.1
numpy version:1.22.3
h2o version:3.30.0.4
sklearn version:1.0.2


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
conn_s3 = dg_s3.Connect('dgdatadump/DS&CA/Pricing/02 Projects/2022/Tel_Reg_2.0/Cancellation Model')

df_raw12 = conn_s3.read('TelReg_Canx_filt_12mo.csv')
df_raw9 = conn_s3.read('TelReg_Canx_filt_9mo.csv')
df_raw6 = conn_s3.read('TelReg_Canx_filt_6mo.csv')
df_raw3 = conn_s3.read('TelReg_Canx_filt_3mo.csv')

In [51]:
df_raw9.columns.to_list()

['offertotalprice',
 'TotalPrice',
 'PricingVersionDescription',
 'PurchasePrice',
 'Fee',
 'PlanLiveCount',
 'PlansActivePast1yrCount',
 'PlansActivePast5yrCount',
 'PlansAcceptedPast1YearCount',
 'PlansAcceptedPast5YearCount',
 'PlansCancelledPast1YearCount',
 'PlansCancelledPast5YearCount',
 'ClaimPast1YearCount',
 'ClaimPast3YearCount',
 'ClaimPast5YearCount',
 'ClaimAmount1YearTotal',
 'ClaimAmount3YearTotal',
 'ClaimAmount5YearTotal',
 'GoodsColour',
 'ClientGroupDesc',
 'ClientAccountDesc',
 'RouteToMarketName',
 'ManufacturerGuaranteeLabourMonths',
 'ManufacturerGuaranteePartsMonths',
 'ApplianceCode',
 'ClientGroupCode',
 'ClientCode',
 'ManufacturerBrandCode',
 'price_diff',
 'price_band',
 'HasPrevRTPPrice',
 'AppAge',
 'relFee',
 'relFeeInc',
 'cancelflag',
 'HPI_AveragePrice',
 'HPI_SalesVolume',
 'ONS_avg_age',
 'ONS_avg_distance_travelled_to_work(km)',
 'ONS_%people_working_FT',
 'ONS_%people_working_49+',
 'ONS_%people_working_sum',
 'ONS_avg_household_size',
 'ONS_n_ho

In [4]:
data_list = ['df_raw3', 'df_raw6', 'df_raw9', 'df_raw12']
data_set = [df_raw3, df_raw6, df_raw9, df_raw12]

In [5]:
# read feature importance performance pickle
feat_imp_performance_df = pd.read_pickle("feat_imp_performance_df.pkl")
# read model performance pickle
performance_df = pd.read_pickle("performance_df.pkl")

In [6]:
# assign value for each row corresponding to dataset
tuned_performance_df = pd.DataFrame(columns =['name', 'varimp', 'feats_stored_gbm', 'perf_metrics','classif_rep', 'xval_auc', 'xval_metrics']
                                    )
tuned_performance_df['name']=data_list

In [7]:
for i in range(len(data_set)):
    print(f'{data_list[i]}: {data_set[i].shape[0]}')

df_raw3: 357058
df_raw6: 270538
df_raw9: 167819
df_raw12: 66895


In [8]:
# for 12/9/6/3-month journey
idx = 2
predictors = performance_df.loc[idx, 'feats_stored_gbm']
response = 'cancelflag'
dataset = performance_df.loc[idx, 'name']
print(f'dataset: {dataset}')
print(f'predictors: {predictors}')
print(f'number of predictors: {len(predictors)}')

dataset: df_raw9
predictors: ['PlansAcceptedPast1YearCount', 'PlanLiveCount', 'AppAge', 'ONS_%people_working_FT', 'ManufacturerBrandCode', 'ClientAccountDesc', 'ONS_avg_age', 'ApplianceCode', 'PlansAcceptedPast5YearCount', 'PurchasePrice', 'ClientGroupDesc', 'GoodsColour', 'ONS_avg_dependent_children_per_family', 'ONS_n_families_per_population', 'ONS_%people_working_49+', 'ONS_avg_bedrooms', 'ONS_avg_rooms', 'relFee', 'HPI_SalesVolume', 'ONS_occupation_rate']
number of predictors: 20


In [9]:
# h2o.shutdown()

In [10]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1-internal" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1-internal+0-adhoc..src); OpenJDK 64-Bit Server VM (build 11.0.9.1-internal+0-adhoc..src, mixed mode)
  Starting server from /home/ubuntu/anaconda3/envs/ahmad_env/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpj3_yngzv
  JVM stdout: /tmp/tmpj3_yngzv/h2o_ubuntu_started_from_python.out
  JVM stderr: /tmp/tmpj3_yngzv/h2o_ubuntu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,"2 years, 3 months and 10 days !!!"
H2O_cluster_name:,H2O_from_python_ubuntu_scop8r
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,29.97 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"accepting new members, healthy"


In [11]:
df = data_set[idx].copy()
# for i in df.columns:
#     df[i] = df[i].astype('category')
    
hf_df = h2o.H2OFrame(df)

hf_train, hf_valid, hf_test = hf_df.split_frame(
    ratios=[0.4,0.3], 
    seed=1234, 
    destination_frames=['train.hex','valid.hex','test.hex']
)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
hf_train.shape, hf_valid.shape, hf_test.shape

((67240, 54), (50342, 54), (50237, 54))

In [14]:
## format response into a categorical/factor for binary classification
hf_df['cancelflag']=hf_df['cancelflag'].asfactor()

hf_train['cancelflag']=hf_train['cancelflag'].asfactor()
hf_test['cancelflag']=hf_test['cancelflag'].asfactor()
hf_valid['cancelflag']=hf_valid['cancelflag'].asfactor()

## Base Model

In [15]:
#We only provide the required parameters, everything else is default
gbm = H2OGradientBoostingEstimator()
gbm.train(x=predictors, y=response, training_frame=hf_train)


y_test = hf_test.as_data_frame()
y_test=y_test['cancelflag']

# classification report
y_true = y_test
y_pred = gbm.predict(hf_test)
y_pred = y_pred.as_data_frame()

print(classification_report(y_true, y_pred['predict'], labels=[0,1]))
print_model_perf_stats(gbm, hf_train, hf_valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
              precision    recall  f1-score   support

           0       0.84      0.58      0.68     40073
           1       0.25      0.57      0.35     10164

    accuracy                           0.58     50237
   macro avg       0.55      0.57      0.52     50237
weighted avg       0.72      0.58      0.62     50237

MCC: 0.20518 / 0.14160
F1: 0.38807 / 0.35752
AUC: 0.65745 / 0.61286
AUC PR: 0.37735 / 0.30999
Accuracy: 0.80579 / 0.80489
Logloss: 0.47701 / 0.48240


In [16]:
perf_base= gbm.model_performance(hf_valid)
print(perf_base.auc())

0.6128554008204605


In [ ]:
# path = 'GBM_cancellation_model_3mo_not_tuned'
# gbm.save_mojo(path)

In [17]:
  gbm_lucky = H2OGradientBoostingEstimator(
    ## more trees is better if the learning rate is small enough 
    ## here, use "more than enough" trees - we have early stopping
    ntrees = 10000,                                                            

    ## smaller learning rate is better (this is a good value for most datasets, but see below for annealing)
    learn_rate = 0.01,                                                         

    ## early stopping once the validation AUC doesn't improve by at least 0.01% for 5 consecutive scoring events
    stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "AUC", 

    ## sample 80% of rows per tree
    sample_rate = 0.8,                                                       

    ## sample 80% of columns per split
    col_sample_rate = 0.8,                                                   

    ## fix a random number generator seed for reproducibility
    seed = 1234,                                                             

    ## score every 10 trees to make early stopping reproducible (it depends on the scoring interval)
    score_tree_interval = 10)

  gbm_lucky.train(x=predictors, y=response, training_frame=hf_train, validation_frame=hf_valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [18]:
perf_lucky = gbm_lucky.model_performance(hf_valid)
print(perf_lucky.auc())

0.6393392538793843


## Hyper-Parameter Search

In [19]:
df = data_set[idx].copy()
response = 'cancelflag'

# convert columns to factors
# for i in data_set[idx].columns[0:]:
#     df[i] = data_set[idx][i].astype('category')

# re train test split
x_train, x_test, y_train, y_test = train_test_split(df[predictors], df[response], test_size = 0.5, random_state = 1234)
train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)

# preserve training & test data's index to be used as a key to merge prediction probability with main dataset 
xtrain_index = x_train.index
xtest_index = x_test.index

# convert to h2o dataframe format
hf_train = h2o.H2OFrame(train)
hf_test = h2o.H2OFrame(test)

# format target to fit the model
hf_train['cancelflag']=hf_train['cancelflag'].asfactor()
hf_test['cancelflag']=hf_test['cancelflag'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


### Find Max Depth

In [20]:
## Depth 10 is usually plenty of depth for most datasets, but you never know

hyper_params = {'max_depth' : [4,6,8,12,16,20]}
#faster for larger datasets
# hyper_params = {'max_depth' : list(range(1,10,2))}

#Build initial GBM Model
gbm_grid = H2OGradientBoostingEstimator(
        # keep_cross_validation_predictions=True, 
        ## more trees is better if the learning rate is small enough 
        ## here, use "more than enough" trees - we have early stopping
        ntrees=10000,
        ## smaller learning rate is better
        ## since we have learning_rate_annealing, we can afford to start with a 
        #bigger learning rate
        learn_rate=0.05,
        ## learning rate annealing: learning_rate shrinks by 1% after every tree 
        ## (use 1.00 to disable, but then lower the learning_rate)
        learn_rate_annealing = 0.99,
        ## sample 80% of rows per tree
        sample_rate = 0.8,
        ## sample 80% of columns per split
        col_sample_rate = 0.8,
        ## fix a random number generator seed for reproducibility
        seed = 1234,
        ## score every 10 trees to make early stopping reproducible 
        #(it depends on the scoring interval)
        score_tree_interval = 10, 
        ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
        #5 consecutive scoring events
        stopping_rounds = 5,
        stopping_metric = "AUC",
        stopping_tolerance = 1e-4)

#Build grid search with previously made GBM and hyper parameters
grid = H2OGridSearch(gbm_grid,hyper_params, grid_id = 'depth_grid', search_criteria = {'strategy': "Cartesian"})

 
#Train grid search
grid.train(x=predictors, y=response, training_frame = hf_train, validation_frame = hf_test)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [21]:
## display the grid search results sorted by increasing logloss 
print(grid)

    max_depth           model_ids              logloss
0          16  depth_grid_model_5  0.46551045770340443
1          20  depth_grid_model_6  0.46904038143734295
2          12  depth_grid_model_4   0.4720088722208109
3           8  depth_grid_model_3  0.48126246799962696
4           6  depth_grid_model_2   0.4844569572760149
5           4  depth_grid_model_1  0.48718218911501737



In [22]:
## sort the grid models by decreasing AUC
sorted_grid = grid.get_grid(sort_by='auc',decreasing=True)
print(sorted_grid)

    max_depth           model_ids                 auc
0          20  depth_grid_model_6   0.683160966007034
1          16  depth_grid_model_5   0.677637975334741
2          12  depth_grid_model_4  0.6597153719144156
3           8  depth_grid_model_3  0.6286159929184442
4           6  depth_grid_model_2  0.6165481721737709
5           4  depth_grid_model_1  0.6074174952535651



--- BREAK --

### Grid Search

In [23]:
max_depths = sorted_grid.sorted_metric_table()['max_depth'][0:3]
new_max = int(max(max_depths, key=int))
new_min = int(min(max_depths, key=int))

print("MaxDepth", new_max)
print("MinDepth", new_min)

MaxDepth 20
MinDepth 12


In [24]:
# create hyperameter and search criteria lists (ranges are inclusive..exclusive))
hyper_params_tune = {'max_depth' : list(range(new_min,new_max+1,1)),
                'sample_rate': [x/100. for x in range(20,101)],
                'col_sample_rate' : [x/100. for x in range(20,101)],
                'col_sample_rate_per_tree': [x/100. for x in range(20,101)],
                'col_sample_rate_change_per_level': [x/100. for x in range(90,111)],
                'min_rows': [2**x for x in range(0,int(math.log(hf_train.nrow,2)-1)+1)],
                'nbins': [2**x for x in range(4,11)],
                'nbins_cats': [2**x for x in range(4,13)],
                'min_split_improvement': [0,1e-8,1e-6,1e-4],
                'histogram_type': ["UniformAdaptive","QuantilesGlobal","RoundRobin"]}
search_criteria_tune = {'strategy': "RandomDiscrete",
                   'max_runtime_secs': 300,  ## limit the runtime to 60 minutes
                   'max_models': 100,  ## build no more than 100 models
                   'seed' : 1234,
                   'stopping_rounds' : 5,
                   'stopping_metric' : "AUC",
                   'stopping_tolerance': 1e-3
                   }

In [38]:
gbm_final_grid = H2OGradientBoostingEstimator(
                    keep_cross_validation_predictions=True, 
                    distribution='bernoulli',
                    ## more trees is better if the learning rate is small enough 
                    ## here, use "more than enough" trees - we have early stopping
                    ntrees=10000,
                    ## smaller learning rate is better
                    ## since we have learning_rate_annealing, we can afford to start with a 
                    #bigger learning rate
                    learn_rate=0.05,
                    ## learning rate annealing: learning_rate shrinks by 1% after every tree 
                    ## (use 1.00 to disable, but then lower the learning_rate)
                    learn_rate_annealing = 0.99,
                    ## score every 10 trees to make early stopping reproducible 
                    #(it depends on the scoring interval)
                    score_tree_interval = 10,
                    ## fix a random number generator seed for reproducibility
                    seed = 1234,
                    ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
                    #5 consecutive scoring events
                    stopping_rounds = 5,
                    stopping_metric = "AUC",
                    stopping_tolerance = 1e-4)
            
#Build grid search with previously made GBM and hyper parameters
final_grid = H2OGridSearch(gbm_final_grid, hyper_params = hyper_params_tune,
                                    grid_id = 'final_grid',
                                    search_criteria = search_criteria_tune)
#Train grid search
final_grid.train(x=predictors, 
           y=response,
           ## early stopping based on timeout (no model should take more than 1 hour - modify as needed)
           max_runtime_secs = 360, 
           training_frame = hf_train,
           validation_frame = hf_valid)

print(final_grid)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%
    col_sample_rate col_sample_rate_change_per_level col_sample_rate_per_tree  \
0               0.5                             1.02                     0.65   
1               0.5                             1.02                     0.65   

  histogram_type max_depth min_rows min_split_improvement nbins nbins_cats  \
0     RoundRobin        19      8.0                1.0E-8   512        512   
1     RoundRobin        19      8.0                1.0E-8   512        512   

  sample_rate           model_ids              logloss  
0        0.64  final_grid_model_2  0.32393161733312825  
1        0.64  final_grid_model_1   0.4550380124243037  



In [39]:
## Sort the grid models by AUC
sorted_final_grid = final_grid.get_grid(sort_by='auc',decreasing=True)

print(sorted_final_grid)

    col_sample_rate col_sample_rate_change_per_level col_sample_rate_per_tree  \
0               0.5                             1.02                     0.65   
1               0.5                             1.02                     0.65   

  histogram_type max_depth min_rows min_split_improvement nbins nbins_cats  \
0     RoundRobin        19      8.0                1.0E-8   512        512   
1     RoundRobin        19      8.0                1.0E-8   512        512   

  sample_rate           model_ids                auc  
0        0.64  final_grid_model_2  0.883938701390189  
1        0.64  final_grid_model_1  0.703833228011264  



## Model Inspection and Final Test Set Scoring

In [40]:
#Get the best model from the list (the model name listed at the top of the table)
best_model = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][0])
performance_best_model = best_model.model_performance(hf_test)
print(performance_best_model.auc())

0.7038443139962371


In [41]:
# inspect the winning model's parameters
params_list = []
for key, value in best_model.params.items():
    params_list.append(str(key)+" = "+str(value['actual']))
params_list

["model_id = {'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': 'final_grid_model_2', 'type': 'Key<Model>', 'URL': '/3/Models/final_grid_model_2'}",
 "training_frame = {'__meta': {'schema_version': 3, 'schema_name': 'FrameKeyV3', 'schema_type': 'Key<Frame>'}, 'name': 'py_8_sid_9510', 'type': 'Key<Frame>', 'URL': '/3/Frames/py_8_sid_9510'}",
 "validation_frame = {'__meta': {'schema_version': 3, 'schema_name': 'FrameKeyV3', 'schema_type': 'Key<Frame>'}, 'name': 'py_7_sid_9510', 'type': 'Key<Frame>', 'URL': '/3/Frames/py_7_sid_9510'}",
 'nfolds = 0',
 'keep_cross_validation_models = True',
 'keep_cross_validation_predictions = True',
 'keep_cross_validation_fold_assignment = False',
 'score_each_iteration = False',
 'score_tree_interval = 10',
 'fold_assignment = AUTO',
 'fold_column = None',
 "response_column = {'__meta': {'schema_version': 3, 'schema_name': 'ColSpecifierV3', 'schema_type': 'VecSpecifier'}, 'column_name': 'cancelflag', 'is

### Cross Validation on the whole dataset

In [42]:
# Build a GBM model on the whole dataset (instead of the 60%) and using internal 5-fold cross-validation (re-using all other parameters including the seed)
gbm = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][0])

#get the parameters from the Random grid search model and modify them slightly
params = gbm.params
new_params = {"nfolds":5, "model_id":None, "training_frame":None, "validation_frame":None, 
              "response_column":None, "ignored_columns":None}
for key in new_params.keys():
    params[key]['actual'] = new_params[key] 
gbm_best = H2OGradientBoostingEstimator()
for key in params.keys():
    if key in dir(gbm_best) and getattr(gbm_best,key) != params[key]['actual']:
        setattr(gbm_best,key,params[key]['actual'])

In [ ]:
# params['class_sampling_factors']

In [ ]:
# df = data_set[idx].copy()
# for i in df.columns:
#     df[i] = df[i].astype('category')
    
# hf_df = h2o.H2OFrame(df)

In [43]:
# train GBM model on the whole dataset
start = time.time()
gbm_best.train(x=predictors, y=response, training_frame=hf_df)
end = time.time()
print(end - start)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
296.66517448425293


In [44]:
# cross validation on the whole dataset
print(gbm_best.cross_validation_metrics_summary())


Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.74399304,0.012591192,0.72649217,0.7352267,0.753498,0.7491968,0.7555516
1,auc,0.72044444,0.003791916,0.7212921,0.7137788,0.7224432,0.72162014,0.7230879
2,aucpr,0.46302915,0.005588553,0.4605558,0.45886308,0.47274774,0.46042606,0.462553
3,err,0.25600696,0.012591192,0.27350783,0.2647733,0.246502,0.25080317,0.24444842
4,err_count,8593.0,436.73962,9206.0,8894.0,8333.0,8353.0,8179.0
5,f0point5,0.41748923,0.012508227,0.40099278,0.4087494,0.43149576,0.42009845,0.42610982
6,f1,0.449871,0.005154655,0.4466883,0.4444722,0.4576635,0.44853768,0.4519933
7,f2,0.4881433,0.00942467,0.5041378,0.48703697,0.48721004,0.48110697,0.4812247
8,lift_top_group,4.4042664,0.055390164,4.416641,4.3123407,4.461831,4.4077277,4.4227896
9,logloss,0.45030206,0.003078745,0.44849443,0.45440605,0.45224738,0.4497662,0.44659624



See the whole table with table.as_data_frame()



### GBM Lucky Performance

In [49]:
# classification report
y_true = y_test
y_pred = gbm_lucky.predict(hf_test)
y_pred = y_pred.as_data_frame()

print(classification_report(y_true, y_pred['predict'], labels=[0,1]))
print_model_perf_stats(gbm_lucky, hf_train, hf_test)

gbm prediction progress: |████████████████████████████████████████████████| 100%
              precision    recall  f1-score   support

           0       0.87      0.60      0.71     66956
           1       0.29      0.65      0.41     16954

    accuracy                           0.61     83910
   macro avg       0.58      0.63      0.56     83910
weighted avg       0.76      0.61      0.65     83910

MCC: 0.24563 / 0.23850
F1: 0.41446 / 0.40886
AUC: 0.69044 / 0.68530
AUC PR: 0.40992 / 0.40474
Accuracy: 0.80922 / 0.80893
Logloss: 0.46406 / 0.46478


### Best Model Performance

In [45]:
y_test = hf_test.as_data_frame()
y_test=y_test['cancelflag']

# classification report
y_true = y_test
y_pred = best_model.predict(hf_test)
y_pred = y_pred.as_data_frame()

print(classification_report(y_true, y_pred['predict'], labels=[0,1]))

gbm prediction progress: |████████████████████████████████████████████████| 100%
              precision    recall  f1-score   support

           0       0.84      0.93      0.88     66956
           1       0.51      0.30      0.38     16954

    accuracy                           0.80     83910
   macro avg       0.68      0.62      0.63     83910
weighted avg       0.77      0.80      0.78     83910



In [46]:
best_model.model_performance(hf_test)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.14324707000228637
RMSE: 0.3784799466316364
LogLoss: 0.455036307115867
Mean Per-Class Error: 0.3534341998548054
AUC: 0.7038443139962371
AUCPR: 0.4387163153467974
Gini: 0.4076886279924743

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1989675573077452: 


,,0,1,Error,Rate
0,0,52747.0,14209.0,0.2122,(14209.0/66956.0)
1,1,8427.0,8527.0,0.4971,(8427.0/16954.0)
2,Total,61174.0,22736.0,0.2698,(22636.0/83910.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.198968,0.429680,230.0
1,max f2,0.094792,0.576171,333.0
2,max f0point5,0.306563,0.450988,161.0
3,max accuracy,0.417924,0.813538,115.0
4,max precision,0.975280,1.000000,0.0
5,max recall,0.020583,1.000000,398.0
6,max specificity,0.975280,1.000000,0.0
7,max absolute_mcc,0.269547,0.287748,181.0
8,max min_per_class_accuracy,0.157093,0.641265,267.0
9,max mean_per_class_accuracy,0.181089,0.646566,244.0



Gains/Lift Table: Avg response rate: 20.20 %, avg score: 17.05 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010011,0.685894,4.295263,4.295263,0.867857,0.836121,0.867857,0.836121,0.042999,0.042999,329.526323,329.526323
1,,2,0.020010,0.562920,3.397833,3.846816,0.686532,0.618607,0.777248,0.727429,0.033974,0.076973,239.783327,284.681550
2,,3,0.030008,0.489134,3.096983,3.596971,0.625745,0.523268,0.726767,0.659402,0.030966,0.107939,209.698345,259.697075
3,,4,0.040007,0.439942,2.849225,3.410090,0.575685,0.462731,0.689008,0.610249,0.028489,0.136428,184.922478,241.008994
4,,5,0.050006,0.404527,2.583769,3.244865,0.522050,0.421090,0.655624,0.572426,0.025835,0.162263,158.376905,224.486515
5,,6,0.100000,0.309409,2.117747,2.681373,0.427890,0.350177,0.541771,0.461315,0.105875,0.268137,111.774678,168.137313
6,,7,0.150006,0.260361,1.683178,2.348615,0.340086,0.282840,0.474537,0.401818,0.084169,0.352306,68.317796,134.861497
7,,8,0.200000,0.229407,1.324919,2.092721,0.267700,0.244221,0.422834,0.362424,0.066238,0.418544,32.491902,109.272148
8,,9,0.300000,0.188351,1.147812,1.777752,0.231915,0.207411,0.359194,0.310753,0.114781,0.533325,14.781173,77.775156
9,,10,0.400000,0.160155,0.949039,1.570573,0.191753,0.173583,0.317334,0.276460,0.094904,0.628229,-5.096143,57.057332


In [47]:
model_perf_stats(best_model, hf_train, hf_test)

['MCC: 0.96043 / 0.28775',
 'F1: 0.96842 / 0.42968',
 'AUC: 0.99879 / 0.70384',
 'AUC PR: 0.99547 / 0.43872',
 'Accuracy: 0.98725 / 0.81354',
 'Logloss: 0.19938 / 0.45504']

In [48]:
for i in range(len(sorted_final_grid.sorted_metric_table()['model_ids'])):
    mod = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][i])
    perf1 = mod.model_performance(hf_train)
    perf2 = mod.model_performance(hf_test)

    auc_diff = (perf1.auc()-perf2.auc())/perf1.auc()
    print(f'{i}: {auc_diff}')
    
    y_pred = mod.predict(hf_test)
    y_pred = y_pred.as_data_frame()

    print(classification_report(y_true, y_pred['predict'], labels=[0,1]))

0: 0.29530261677309344
gbm prediction progress: |████████████████████████████████████████████████| 100%
              precision    recall  f1-score   support

           0       0.84      0.93      0.88     66956
           1       0.51      0.30      0.38     16954

    accuracy                           0.80     83910
   macro avg       0.68      0.62      0.63     83910
weighted avg       0.77      0.80      0.78     83910

1: 0.29531096643744226
gbm prediction progress: |████████████████████████████████████████████████| 100%
              precision    recall  f1-score   support

           0       0.86      0.78      0.82     66956
           1       0.37      0.51      0.43     16954

    accuracy                           0.73     83910
   macro avg       0.62      0.65      0.63     83910
weighted avg       0.76      0.73      0.74     83910



In [ ]:
# best_model = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][0])
# model_perf_stats(best_model, hf_train, hf_test)

In [ ]:
# train GBM model on training dataset
start = time.time()
gbm_best.train(x=predictors, y=response, training_frame=hf_train)
end = time.time()
print(end - start)

In [ ]:
# variable importance
varimp = gbm_best.varimp(use_pandas=True).sort_values('scaled_importance', ascending=False)
feats_stored_gbm = list(varimp['variable'])
#perf metrics
perf_metrics = model_perf_stats(gbm_best, hf_train, hf_test)

# classification report
y_pred = gbm_best.predict(hf_test)
y_pred = y_pred.as_data_frame()
classif_rep = classification_report(y_true, y_pred['predict'], labels=[0,1])

# AUC of cross-validated holdout predictions
xval_auc =  f'AUC of cross-validated holdout predictions: {gbm_best.auc(xval=True)}'

# cross-validation metrics summary
xval_metrics = gbm_best.cross_validation_metrics_summary()

In [ ]:
# # variable importance
# varimp = best_model.varimp(use_pandas=True).sort_values('scaled_importance', ascending=False)
# feats_stored_gbm = list(varimp['variable'])
# #perf metrics
# perf_metrics = model_perf_stats(best_model, hf_train, hf_test)

# # classification report
# y_pred = best_model.predict(hf_test)
# y_pred = y_pred.as_data_frame()
# classif_rep = classification_report(y_true, y_pred['predict'], labels=[0,1])
# # AUC of cross-validated holdout predictions
# xval_auc =  f'AUC of cross-validated holdout predictions: {best_model.auc(xval=True)}'
# # cross-validation metrics summary
# xval_metrics = best_model.cross_validation_metrics_summary()

In [ ]:
best_model.auc(xval=True)

In [ ]:
# assign value to row
tuned_performance_df.loc[idx,1:] = [varimp, feats_stored_gbm, perf_metrics, classif_rep, xval_auc, xval_metrics]

In [ ]:
# 12 month
for j in ['name','feats_stored_gbm','perf_metrics','classif_rep', 'xval_auc']:

    print(tuned_performance_df.loc[idx,j])

gbm_best12 = gbm_best

In [ ]:
# # 9 month
# for j in ['name','feats_stored_gbm','perf_metrics','classif_rep', 'xval_auc']:

#     print(tuned_performance_df.loc[idx,j])

# gbm_best9 = best_model

In [ ]:
# # 6 month
# for j in ['name','feats_stored_gbm','perf_metrics','classif_rep', 'xval_auc']:

#     print(tuned_performance_df.loc[idx,j])
    
# gbm_best6 = best_model

In [ ]:
# # 3 month
# for j in ['name','feats_stored_gbm','perf_metrics','classif_rep', 'xval_auc']:

#     print(tuned_performance_df.loc[idx,j])

# gbm_best3 = best_model

In [ ]:
# print performance metrics for all cancellation cohorts
for i in range(len(tuned_performance_df)):
    for j in ['name','feats_stored_gbm','perf_metrics','classif_rep', 'xval_auc']:

        print(tuned_performance_df.loc[i,j])

In [ ]:
# print cross-validated performance metrics for all cancellation cohorts
for i in range(len(tuned_performance_df)):
    print(tuned_performance_df.loc[i,'name'])
    print(tuned_performance_df.loc[i,'xval_metrics'])

In [ ]:
# # Extract feature interactions:
# feature_interactions = gbm_best.feature_interaction()

## Save Performance

In [ ]:
# # save performance metric to pickle
# pd.to_pickle(tuned_performance_df, "tuned_performance_df.pkl")

## Save Models

In [50]:
# # saving down the model
# path = 'GBM_cancellation_model_3mo'
# gbm_best3.save_mojo(path)

# path = 'GBM_cancellation_model_6mo'
# gbm_best6.save_mojo(path)

# path = 'GBM_cancellation_model_9mo'
# gbm_best9.save_mojo(path)

# path = 'GBM_cancellation_model_12mo'
# gbm_best12.save_mojo(path)
path = 'GBM_cancellation_model_9mo_lucky'
gbm_lucky.save_mojo(path)

'/home/ubuntu/Tel_Reg_2.0/Cancellation_Model/GBM_cancellation_model_9mo_lucky/GBM_model_python_1662980792682_56.zip'

## Resampling

In [ ]:
# df = df_raw3
# class_counts = df['cancelflag'].value_counts()
# class_counts

In [ ]:
# minority_label = 1
# majority_label = 0
# class_counts[minority_label] # count of minority class 
# sampled_df = imbalanced_undersample(df, 'cancelflag', minority_label, majority_label)
# sampled_df['cancelflag'].value_counts()